In [ ]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent / "src"))

import pandas as pd
import numpy as np
import plotly.express as px


In [ ]:
period = pd.Period('2023')

In [ ]:
from holiday import pl_bd
vac_days = 30
bdays = pd.bdate_range(start=period.start_time, end=period.end_time, freq=pl_bd).size
vac_coeff = (bdays - vac_days) / bdays

In [ ]:
from private import email_dict, team_emails, team_contracts
from holiday import pl_bd

df = pd.read_csv('commits.csv', index_col=0, parse_dates=['date'])
df = df.replace({'email': email_dict})
df = df.loc[(df['date'] >= period.start_time) & (df['date'] <= period.end_time)]
df['mod'] = df['ins'] + df['del']

df = df.loc[~df['path'].astype(str).str.startswith('wp-content/uploads/')]
df = df.loc[~df['path'].astype(str).str.startswith('wp-content/plugins/')]
df = df.loc[~df['path'].astype(str).str.startswith('wp-content/themes/')]
df = df.loc[~df['path'].astype(str).str.startswith('wp-content/wflogs/')]
df = df.loc[~df['path'].astype(str).str.startswith('wp-includes/')]
df = df.loc[~df['path'].astype(str).str.startswith('wp-admin/')]
df = df.loc[~df['path'].astype(str).str.startswith('wp-login.php')]
df = df.loc[~df['path'].astype(str).str.startswith('dm/src/main/resources/VAADIN/themes/at/styles.css')]
df = df.loc[~df['path'].astype(str).str.startswith('wp-includes/js/plupload/moxie.js')]
df = df.loc[~df['path'].astype(str).str.startswith('grafana/longevity')]
df = df.loc[~df["path"].astype(str).str.startswith("build/")]
df = df.loc[~df['path'].astype(str).str.endswith('package-lock.json')]
df = df.loc[~df['path'].astype(str).str.endswith('yarn.lock')]
df = df.loc[~df['path'].astype(str).str.endswith('pl_PL.po')]
df = df.loc[~df['path'].astype(str).str.endswith('jquery.js')]
df = df.loc[~df['path'].astype(str).str.endswith('bootstrap.min.css')]
df = df.loc[~df['path'].astype(str).str.endswith('FIX44.xml')]
df = df.loc[~df["path"].astype(str).str.endswith("gradlew")]
df = df.loc[~df["path"].astype(str).str.endswith("gradlew.bat")]
df = df.loc[~df["path"].astype(str).str.endswith(".png")]
df = df.loc[~df["path"].astype(str).str.endswith(".jar")]
df = df.loc[~df["path"].astype(str).str.endswith(".log")]
df = df.loc[~df["path"].astype(str).str.endswith(".sql")]
df = df.loc[~df["path"].astype(str).str.endswith(".xsl")]
df = df.loc[~df["path"].astype(str).str.endswith(".backup")]
df = df.loc[~df["path"].astype(str).str.endswith(".back")]
df = df.loc[~df["path"].astype(str).str.endswith(".snap")]
df = df.loc[~df["path"].astype(str).str.endswith(".tfstate")]
df = df.loc[(~df["path"].astype(str).str.endswith("custom.css")) | (df['del'] != 0)]
df = df.loc[~df['path'].astype(str).str.contains('realm.*json')]
df = df.loc[~df['path'].astype(str).str.contains('/fontawesome/')]
df = df.loc[~df['path'].astype(str).str.contains('/generated/')]
df = df.loc[~df['path'].astype(str).str.contains('/grafana/.*json')]

df = df.groupby('email')['mod'].agg('sum').reset_index()
df = df[df['email'].isin(team_emails)].sort_values('mod')
df = df.set_index('email')

In [ ]:
tc = pd.DataFrame(team_contracts, columns=['email', 'start', 'end'])
tc['start_p'] = period.start_time
tc['end_p'] = period.end_time
tc['start_w'] = tc[['start', 'start_p']].max(axis=1)
tc['end_w'] = tc[['end', 'end_p']].min(axis=1)
tc['bdays'] = tc.apply(lambda row: pd.bdate_range(start=row['start_w'], end=row['end_w'], freq=pl_bd).size, axis=1)
tc['wdays'] = tc['bdays'] * vac_coeff
tc = tc.drop(['start', 'end', 'start_p', 'end_p', 'start_w', 'end_w', 'bdays'], axis=1)
tc = tc.set_index('email')

In [ ]:
df = df.merge(tc, left_index=True, right_index=True)
df['lpd'] = df['mod'].div(df['wdays']).round()
df.sort_values('lpd')